# 개요

- 업스트림 모델 주소
    - 허깅페이스
        - url
            - https://huggingface.co/skt/kogpt2-base-v2
        - repo
            - skt/kogpt2-base-v2
        - git
            - https://github.com/SKT-AI/KoGPT2
    - gpt
        - 문장생성, 번역등에 강점을가진 모델
- 목표
    - 전이학습 기법을 활용, 사전학습된 업스트림 모델을 가져와서 별도 학습 없이(제로샷러닝) 문장 생성이라는 기능으로 활용
    - 문장 생성 서비스 구현
    - 방식
        - 입력
            - 적당량 입력(텍스트) 주입
            - 텍스트 직접입력
            - STT 입력
            - 영상/이미지 입력 -> 텍스트 추출 -> 입력
        - 출력
            - 문장생성 응답

- 사전 크기
    - 51,200 자 (토큰수)

- 학습
    - 특정 목적을 위해 사전학습된 모델을 그대로 사용
        - 제로샷
    - 차후 필요시
        - 추가 데이터 학습(사내 데이터, 외부 공개 불가한 데이터, ..)
        - 특정 데이터에 최적화된 모델로 가중치 조정
            - 파인튜닝의 프럼프트 러닝으로 확장

- 구현
    - 데모
        - grdio, streamlit 을 통해서 서비스 구현
    - 서비스
        - flask, fastpi 구현(웹)

# 트랜스포머 설치

- 트랜스포머 기반 (가장 먼저 진행한 코드)

In [1]:
!pip install -q transformers

# skt/kogpt2-base-v2 로드

- repo
    - 'skt/kogpt2-base-v2'
- load
    - 토크나이저
        - 텍스트 -> 사전화 -> 백터화 -> 패딩 -> 임베딩 전과정 지원
        - 텍스트 입력 -> 백터화 제공
        - 예측값(백터) -> 디코딩 -> 텍스트 제공
        - 단, 제공되는 토크나이저 외에 다른 토크나이저 사용시 -> 별도 학습이 필요
    - 모델
        - 텍스트 입력 -> 텍스트 생성

In [3]:
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
# 토치로 만들어진 모듈 => 텐서로 변환 처리 사용

# QnA 토치 버전으로 git에 공개된 코드로도 진행

In [4]:
repo      = 'skt/kogpt2-base-v2' # 모델의 저장소 주소
tokenizer = AutoTokenizer.from_pretrained( repo ) # 토크나이저 획득
model     = TFGPT2LMHeadModel.from_pretrained( repo, from_pt=True ) # 모델 획득

# 토치 모델을 텐서모델로 변경하는 과정에서 특정 기능에 제한이 있거나 등등 제약사항이 존재할수 있다

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

# 문장 생성 테스트 (다운스트림의 목적)

In [5]:
import tensorflow as tf

In [9]:
# 1. 입력 받을 문장을 담을 변수
text = None
# 2. 사용자가 z 만 입력할때까지 사용자는 무한 대기, 초기 문장입력을 기다린다
while True:
    # 2-1. 사용자 입력 대기
    # 오늘 저녁은 무엇을 먹을까
    text = input('작성하고 싶은 글의 초기 문장을 제시하세요\n').strip()
    # 2-2. 탈출코드, 입력값이 z면 종료
    if text == 'z':break
    # 2-3. 비워있는 문자열 입력 -> 다시입력
    if not text:
        print('정확하게 입력하세요')
        continue

    # 2-4. 텍스트 -> 토크나이저 주입 -> 인코딩 -> 백터화
    input_vector = tokenizer.encode( text )
    print( input_vector, type(input_vector) )

    # 2-5. 백터화(토큰값 기준으로 배치) -> 모델에 적합한 형태로 백터 변환 -> 텐서변환, 차원변환
    input_vector = tf.convert_to_tensor( [ input_vector ] )
    print( input_vector, type(input_vector) )

    # 2-6. 모델을 이용하여 문장 생성
    output = model.generate( input_vector,
                    max_length = 64,  # 생성되는 최대 토큰수 제한 (gpt 토큰제한)
                    use_cache  = True,# 캐싱을 활용, 동일 입력에 대한 동일 응답 구성-> 빠른대답
                    repetition_penalty = 2.0 # 테스트 하면서 진행(감도 조정), 반복적 생성 문구 패널티 부여
                    )
    # 2.7. output(텐서) -> 배열변환(백터화) -> 디코딩 -> 텍스트 출력
    output_vector = output.numpy().tolist()[0]
    output_text   = tokenizer.decode( output_vector )
    print( f'\n{output_text}\n' )

작성하고 싶은 글의 초기 문장을 제시하세요
오늘 저녁은 무엇을 먹을까
[10070, 17969, 8135, 22375, 17003, 6969] <class 'list'>
tf.Tensor([[10070 17969  8135 22375 17003  6969]], shape=(1, 6), dtype=int32) <class 'tensorflow.python.framework.ops.EagerTensor'>

오늘 저녁은 무엇을 먹을까 고민하다가
그냥 먹기로 했어요!
이렇게 맛있는 #오뎅탕을 먹고 싶었는데...
그래서 이번에 오뎅이랑 함께 먹어봤는데요
맛있어서 좋았어요.
그리고 이렇게 나온 메뉴는 역시나 맛있어서
다음에 또 와

작성하고 싶은 글의 초기 문장을 제시하세요
근육이 커지기 위해서는
[33245, 10114, 12748, 11357] <class 'list'>
tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32) <class 'tensorflow.python.framework.ops.EagerTensor'>

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐

작성하고 싶은 글의 초기 문장을 제시하세요
맛있는 커피는 
[48541, 10114, 19571] <class 'list'>
tf.Tensor([[48541 10114 19571]], shape=(1, 3), dtype=int32) <class 'tensorflow.python.framework.ops.EagerTensor'>

맛있는 커피는 물론 커피와 함께 즐길 수 있는 다양한 메뉴를 선보이고 있다.
특히, ‘커피 한잔의 행복’이라는 콘셉트로 기획된 이번 행사는 오

- 문제점
    - 세팅된 토큰수 만큼 문장을 생성하고 나서 응답한다
        - 대기시간이 김
    - 제시된 문장의 다은 토큰을 예측해서 제시
        - 제시된 토큰이 가장 중요도(추천순) 높은값인지, 특정 범위내에 랜덤하게 선택된것인지 애매함
            - 선택 방식 여러 기준이 존재

# 문장 생성 테스트 -> 한 토큰씩 계속 응답하는 방식 업그레이드

In [15]:
import numpy as np

text         = '공부를 잘하기 위해서는'
input_vector = tokenizer.encode( text )
input        = np.array( [input_vector] ) # 2차원 배열
output       = model(input) # 모델에 입력 데이터 주입

# (문장의수, 토큰수, 토큰별 데이터 사이즈 )
# kogpt2의 사전의 개수가 51200 -> 토큰을 원-핫인코딩 표현 -> 토큰당 51200의 크기를 가짐
output.logits.shape

TensorShape([1, 4, 51200])

In [16]:
# 실습 1분 output.logits의 마지막 토큰 획득
# 마지맋 토큰 -> 마지막 단어 -> 다음 단어를 추론(예측) 기본 방식
output.logits[0, -1].shape

TensorShape([51200])

In [19]:
# 마지막 토큰을 기점 -> 문장 생성 -> 다음 토큰 예측(추론) 수행 -> 상위 top 5 추출
top5 = tf.math.top_k( output.logits[0, -1], k=5)

print( text, top5.indices.numpy())
for idx in top5.indices.numpy():
    '''
        공부를 잘하기 위해서는 먼저
        공부를 잘하기 위해서는 무엇보다
        공부를 잘하기 위해서는 우선
        공부를 잘하기 위해서는 어떻게
        공부를 잘하기 위해서는 자신의

        대량의 학습(대량의 텍스트 입력) -> 위해서는 다음에 먼저, 무엇보다, .. 등이 중요하게 많이 언급되었다
    '''
    print( idx, tokenizer.decode( idx ) )



공부를 잘하기 위해서는 [10635 23879 12201 11649  9745]
10635 먼저
23879 무엇보다
12201 우선
11649 어떻게
9745 자신의


- 다음 문장 선택
    - 중요도가 가장 높은 토큰 선택
        - 기본 옵션은 정확하게 전달
        - 창의성 부여 -> 사용자에서 선택옵션 부여
    - 특정 랭킹안에든 토큰들 중에서 선택
        - 순환 선택,..
    - **특정 랭킹**안에든 토큰들 중에서 **램덤 선택**
        - top5 후보들중에서 랜덤선택 -> 창의성부여
    - ...

In [22]:
import random

# 대상 기준 랜덤 선택
random.choice( top5.indices.numpy() )

23879

In [24]:
# 문장 제시
# 이어지는 문장을 완성 -> 30토큰 제한 -> 셀 실행 종료
# 마지막 토큰의 다은 토큰은 추천값 상위 5개만 추출해서 랜덤 선택 진행
# 선택된 토큰을 바로 출력
# 실습 5분

text         = '공부를 잘하기 위해서는'
input_vector = tokenizer.encode( text )
start_len    = len(input_vector)
END_MAX_LEN  = 30
print( f'{text}\t')

while len(input_vector) < start_len+END_MAX_LEN:
    # 제시된 문장의 토큰수 + 추가된 문장의 토큰수 => 33 작으면 반복
    input    = np.array( [input_vector] ) # 2차원 배열
    output   = model(input) # 모델에 입력 데이터 주입
    top5     = tf.math.top_k( output.logits[0, -1], k=5)# 탑 5 획득
    token_id = random.choice( top5.indices.numpy() )    # 탑 5 에서 랜덤 선택
    input_vector.append( token_id ) # 새로 생성된 토큰
    print( tokenizer.decode(token_id), end=' ' )

공부를 잘하기 위해서는	
우선 자신이 하고 싶은 공부를 잘하는 게 무엇보다 중요합니다.
 또 자신의 진 로가 무엇인지 정확히 파악하고 이를 위한 준비를 철저히 해야 할 것입니다.
 또 자기 만의 독특한 강 점을 살 